In [1]:
%cd drive/My\ Drive/MinorProject/Mask_RCNN/

/content/drive/My Drive/MinorProject/Mask_RCNN


In [4]:
!git init

Reinitialized existing Git repository in /content/drive/My Drive/MinorProject/Mask_RCNN/.git/


In [5]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   mrcnn/model.py
	modified:   mrcnn/visualize.py
	modified:   samples/balloon/inspect_balloon_data.ipynb
	modified:   samples/demo.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	samples/satellite/

no changes added to commit (use "git add" and/or "git commit -a")


In [0]:
!

In [3]:
%ls

assets/    images/      mask_rcnn_coco.h5    requirements.txt
build/     LICENSE      mask_rcnn.egg-info/  samples/
datasets/  logs/        mrcnn/               setup.cfg
dist/      MANIFEST.in  README.md            setup.py


In [0]:
# cd to the folder in drive where this file is present
%cd drive/My\ Drive/MinorProject/Mask_RCNN/samples/satellite/


/content/drive/My Drive/MinorProject/Mask_RCNN/samples/satellite


In [0]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw

In [0]:
!ls

'Copy of satellite.py'	        inspect_satellite_model.ipynb   satellite.ipynb
 inspect_satellite_data.ipynb   __pycache__		        satellite.py


In [0]:

ROOT_DIR = os.getcwd()
print(ROOT_DIR)
if ROOT_DIR.endswith("samples/satellite"):
  # Go up two levels to the repo root
  ROOT_DIR = os.path.dirname(os.path.dirname(ROOT_DIR))

print(ROOT_DIR)

/content/drive/My Drive/MinorProject/Mask_RCNN/samples/satellite
/content/drive/My Drive/MinorProject/Mask_RCNN


In [0]:
# Import Mask RCNN (mrcnn folder)
sys.path.append(ROOT_DIR+"/mrcnn/")
from config import Config
import utils
import model as modellib

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Using TensorFlow backend.


In [0]:
# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
print(COCO_WEIGHTS_PATH)

/content/drive/My Drive/MinorProject/Mask_RCNN/mask_rcnn_coco.h5


In [0]:
# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")
print(DEFAULT_LOGS_DIR)

/content/drive/My Drive/MinorProject/Mask_RCNN/logs


In [0]:
############################################################
#  Configurations
############################################################


class SatelliteConfig(Config):
  """Configuration for training on the toy  dataset.
  Derives from the base Config class and overrides some values.
  """
  # Give the configuration a recognizable name
  NAME = "satellite"

  # We use a GPU with 12GB memory, which can fit two images.
  # Adjust down if you use a smaller GPU.
  IMAGES_PER_GPU = 2

  # Number of classes (including background)
  NUM_CLASSES = 1 + 3  # Background + {water,built_up,green_cover}

  # Number of training steps per epoch
  STEPS_PER_EPOCH = 100

  # Skip detections with < 90% confidence
  DETECTION_MIN_CONFIDENCE = 0.9


In [0]:
############################################################
#  Dataset
############################################################

class SatelliteDataset(utils.Dataset):

  """  init variables of Dataset
    self._image_ids = []
    self.image_info = []    
    # Background is always the first class
    self.class_info = [{"source": "", "id": 0, "name": "BG"}]
    self.source_class_ids = {}

  """



  def load_satellite(self,dataset_dir,subset):
    """Load a subset of the satellite dataset.
    dataset_dir: Root directory of the dataset.
    type: type to load: train or validation
    """
    # Add classes. We have only three class to add.
    self.add_class("satellite", 1, "water")  #  def add_class(self, source, class_id, class_name):
    self.add_class("satellite", 2, "built_up")
    self.add_class("satellite", 3, "green_cover")

    dataset_dir = os.path.join(dataset_dir, subset)
    # annotations = json.load(open('Anmol/BATCH_1_100_150.json'))
    if subset == "val":
      annotations = json.load(open(os.path.join(dataset_dir, "BATCH_1_100_150.json")))
    if subset == "train":
      annotations = json.load(open(os.path.join(dataset_dir, "train_BATCH_1_150_300.json")))
    images_data = annotations['_via_img_metadata']

    for image_id, image_data in images_data.items():
      regions = image_data['regions']        # regions per image
      image_path = os.path.join(dataset_dir, image_data['filename'])
      width = 500
      height = 500
      if "_450" in image_data['filename']:   # override variables if different sized
        height = 450
        width = 450
      # filename : image_data['filename']

      polygons = [r['shape_attributes'] for r in regions]
      # 
      # print(image_data['filename'])
      # Class ids corresponding to each each polygon
      class_ids = [r['region_attributes'][""] for r in regions]
      int_class_ids =[]
      for ci in class_ids:
  #         print(ci)
        if ci == 'greencover':
          int_class_ids.append(3)
        elif ci == 'buildup':
          int_class_ids.append(2)
        else:
          int_class_ids.append(1) # water body




        # add_image(self, source, image_id, path, **kwargs):

      self.add_image( "satellite",
                      image_id = image_data['filename'],
                      path = image_path,
                      width=width, height=height,
                      polygons=polygons,
                      class_ids = int_class_ids)
        


In [0]:
  def load_mask(self,image_id):
    """Generate instance masks for an image.
    Returns:
    masks: A bool array of shape [height, width, instance count] with
        one mask per instance.
    class_ids: a 1D array of class IDs of the instance masks.
    """
    image_info = self.image_info[image_id]

    # Convert polygons to a bitmap mask of shape
    # [height, width, instance_count]
    info = self.image_info[image_id]

    #length of polygons = number of regions
    mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                    dtype=np.uint8)
    for i, p in enumerate(info["polygons"]):
      # Get indexes of pixels inside the polygon and set them to 1
      if p['name'] == 'rect':
        p['all_points_x'] = [p['x']]
        p['all_points_y'] = [p['y']]
      rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
      mask[rr, cc, i] = 1

    # Return mask, and array of class IDs of each instance. Since we have
    # one class ID only, we return an array of 1s          #earlier

    # Return mask, and array of class IDs of each instance.

    
    return mask, info['class_ids']         #np.ones([mask.shape[-1]], dtype=np.int32)


In [0]:
  def image_reference(self, image_id):
    """Return the path of the image."""
    info = self.image_info[image_id]
    if info["source"] == "satellite":
      return info["path"]
    else:
      super(self.__class__, self).image_reference(image_id)

In [0]:
  def train(model):
    # train the model

    #Training dataset
    dataset_train = SatelliteDataset()
    dataset_train.load_sattelite(args.dataset,"train")
    dataset_train.prepare()

    # Need to create a proper split and merge JSON files, cautions : annotation tags
    # Validation dataset
    dataset_val = SatelliteDataset()
    dataset_val.load_sattelite(args.dataset,"val")
    dataset_cal.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=30,
                layers='heads')
    

    

In [0]:

############################################################
#  Training
############################################################

if __name__ == '__main__':
  import argparse

  # Parse command line arguments
  parser = argparse.ArgumentParser(
      description='Train Mask R-CNN to detect green cover.')
  parser.add_argument("command",
                      metavar="<command>",
                      help="'train' or 'splash'")
  parser.add_argument('--dataset', required=False,
                      metavar="/path/to/balloon/dataset/",
                      help='Directory of the Balloon dataset')
  parser.add_argument('--weights', required=True,
                      metavar="/path/to/weights.h5",
                      help="Path to weights .h5 file or 'coco'")
  parser.add_argument('--logs', required=False,
                      default=DEFAULT_LOGS_DIR,
                      metavar="/path/to/logs/",
                      help='Logs and checkpoints directory (default=logs/)')
  parser.add_argument('--image', required=False,
                      metavar="path or URL to image",
                      help='Image to apply the color splash effect on')
  parser.add_argument('--video', required=False,
                      metavar="path or URL to video",
                      help='Video to apply the color splash effect on')
  args = parser.parse_args()

  # Validate arguments
  if args.command == "train":
    assert args.dataset, "Argument --dataset is required for training"
  elif args.command == "splash":
    assert args.image or args.video,\
              "Provide --image or --video to apply color splash"

  print("Weights: ", args.weights)
  print("Dataset: ", args.dataset)
  print("Logs: ", args.logs)

  # Configurations
  if args.command == "train":
    config = SatelliteConfig()
  else:
    class InferenceConfig(SatelliteConfig):
      # Set batch size to 1 since we'll be running inference on
      # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
      GPU_COUNT = 1
      IMAGES_PER_GPU = 1
    config = InferenceConfig()
  config.display() 

  # Create model
  if args.command == "train":
    model = modellib.MaskRCNN(mode="training", config=config,
                              model_dir=args.logs)
  else:
    model = modellib.MaskRCNN(mode="inference", config=config,
                              model_dir=args.logs)
    
  # Select weights file to load
  if args.weights.lower() == "coco":
    weights_path = COCO_WEIGHTS_PATH
    print("Chi : found pretrained weights")
    # Download weights file
    if not os.path.exists(weights_path):
      utils.download_trained_weights(weights_path)
  elif args.weights.lower() == "last":
    # Find last trained weights
    weights_path = model.find_last()[1]
  elif args.weights.lower() == "imagenet":
    # Start from ImageNet trained weights
    weights_path = model.get_imagenet_weights()
  else:
    weights_path = args.weights


  # Load weights
  print("Loading weights ", weights_path)
  if args.weights.lower() == "coco":
    # Exclude the last layers because they require a matching
    # number of classes
    model.load_weights(weights_path, by_name=True, exclude=[
        "mrcnn_class_logits", "mrcnn_bbox_fc",
        "mrcnn_bbox", "mrcnn_mask"])
  else:
    model.load_weights(weights_path, by_name=True)    

  # Train or evaluate
  if args.command == "train":
    train(model)
  elif args.command == "splash":
    detect_and_color_splash(model, image_path=args.image,
                            video_path=args.video)
  else:
    print("'{}' is not recognized. "
          "Use 'train' or 'splash'".format(args.command))

usage: ipykernel_launcher.py [-h] [--dataset /path/to/balloon/dataset/]
                             --weights /path/to/weights.h5
                             [--logs /path/to/logs/]
                             [--image path or URL to image]
                             [--video path or URL to video]
                             <command>
ipykernel_launcher.py: error: the following arguments are required: --weights


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [0]:
!python3 satellite.py train --dataset=/content/drive/My\ Drive/MinorProject/Mask_RCNN/datasets/ --weights=last

/content/drive/My Drive/MinorProject/Mask_RCNN/samples/satellite
/content/drive/My Drive/MinorProject/Mask_RCNN
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/l

In [0]:
import tensorflow as tf 
print(tf.__version__)

2.2.0-rc3


In [0]:

!pip3 install tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.2MB 54kB/s 
     |████████████████████████████████| 3.2MB 35.5MB/s 
     |████████████████████████████████| 368kB 34.5MB/s 
ERROR: tensorflow 2.2.0rc3 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 2.2.0rc3 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0


In [0]:
import keras
print(keras.__version__)

2.3.1


Using TensorFlow backend.


In [0]:
!pip3 install keras==2.1.0

     |████████████████████████████████| 307kB 3.2MB/s 
ERROR: textgenrnn 1.4.1 has requirement keras>=2.1.5, but you'll have keras 2.1.0 which is incompatible.
  Found existing installation: Keras 2.3.1
    Uninstalling Keras-2.3.1:
      Successfully uninstalled Keras-2.3.1


In [0]:
import tensorflow
print(tensorflow.__version__)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

1.13.1
